In [71]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt       # чтобы выводить промежуточные фото в jupyter

# Список всех настроечных параметров/констант
WORK_DIR = 'pass_photos'
TEST_FILE = 'pass_photos/1.jpeg'
IMG_HEIGHT = 1000            # требуемый размер фото для нормализации всех изображений
IMG_WIDTH = 600              # т.к. в задачу входит прочитать только ФИО, обрезаю серию/номер чтобы не усложнять распознавание
INDENT_LEFT = 200            # обрезаем фото
INDENT_BOTTOM = 200           # обрезаем нижние поля
INDENT_TOP = 40

In [72]:
# Функция для получения списка файлов из каталога с фотографиями (как в task_1 и task_2)
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [73]:
# Подготовка изображений для распознавания текста
def normalize_image(image: str):    
    img = cv2.imread(image)
    
    # нормализуем фото к нужному размеру
    height, width, channels = img.shape
    resize_scale = IMG_HEIGHT / height       # получаем коэффициент масштабирования изображения
    img_width = int(width * resize_scale)    # высчитываем от этого коэффициента новую ширину
    img = cv2.resize(img, (img_width, IMG_HEIGHT))
    
    # обрезаем паспорт до страницы с фото
    x0 = INDENT_LEFT                            # отступ слева, т.к. корочка и фото нам не важны
    y0 = IMG_HEIGHT // 2 + INDENT_TOP           # обрезка сверху, т.к. верхняя страница с местом выдачи нам не важна 
    x1 = img_width if img_width < IMG_WIDTH else IMG_WIDTH     # обрезаем все лишнее справа, если есть разворот с пропиской
    y1 = IMG_HEIGHT - INDENT_BOTTOM
    img = img[y0:y1, x0:x1]
    
    # обесцвечиваем и пытаемся снизить шум с помощью размытия
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
    img = cv2.GaussianBlur(img, (5,5), 0)         # коэффициент размытия подобран вручную
    
    
    # одно изображение используем для распознавания блоков текста. очередность преобраозвания найдена методом тыка
    kernel = np.ones((5,5), 'uint8')
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    img_block = cv2.erode(img, kernel, iterations=1)
    #img_block = cv2.dilate(img_block, kernel, iterations=1)
    _, img_block = cv2.threshold(img_block, 0, 255, cv2.THRESH_OTSU, cv2.THRESH_BINARY_INV)
    img_block = cv2.morphologyEx(img_block, cv2.MORPH_OPEN, kernel, iterations=1)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    # второе изображение используем для распознавания букв внутри блоков
    # при коэффициенте 3 - лучше распознается Васлевский, при 5 - Соколов и Юмакаева
    img_symbol = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 2)
    

    return img_block, img_symbol

In [74]:
# Выделяем элементы текста из изображения
def search_text(blocks, symbols):
    # получаем контуры пятен на изображении
    contours, hierarchy = cv2.findContours(blocks, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    output = blocks.copy()
    
    print(f'Count of counotus: {len(contours)}')
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        # print("R", idx, x, y, w, h, cv2.contourArea(contour), hierarchy[0][idx])
        # hierarchy[i][0]: следующий контур текущего уровня
        # hierarchy[i][1]: предыдущий контур текущего уровня
        # hierarchy[i][2]: первый вложенный элемент
        # hierarchy[i][3]: родительский элемент
        if hierarchy[0][idx][3] == 0:
            cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)
    #cv2.imshow("Input", image)
    cv2.imshow("Output", output)
    cv2.waitKey(0)

In [75]:
# Запускаем цикл по всем фото в рабочей папке
passports = get_files(WORK_DIR)
# col = 5    # количество колонок таблицы
for i, p in enumerate(passports):
    print(f'==== Image {i}.jpg =====')
    blocks, symbols = normalize_image(p)
    cv2.imwrite(f'pass_temp/{i}_blocs.jpg', blocks)
    cv2.imwrite(f'pass_temp/{i}_symbols.jpg', symbols)
    search_text(blocks, symbols)
    
    
    
    
    #plt.subplot(len(passports)//col+1, col, i+1)    # высчитываем высоту таблицы от количества колонок и общего размера списка
    #plt.imshow(image, 'gray')
    # plt.title(p)
    #plt.xticks([]),plt.yticks([])
#plt.show()

==== Image 0.jpg =====
Count of counotus: 18
==== Image 1.jpg =====
Count of counotus: 16
==== Image 2.jpg =====
Count of counotus: 16
==== Image 3.jpg =====
Count of counotus: 16
==== Image 4.jpg =====
Count of counotus: 23
==== Image 5.jpg =====
Count of counotus: 17
==== Image 6.jpg =====
Count of counotus: 17
==== Image 7.jpg =====
Count of counotus: 18
==== Image 8.jpg =====
Count of counotus: 17
==== Image 9.jpg =====
Count of counotus: 15
